In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from icecream import ic
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from xgboost import XGBClassifier

import random
import numpy as np
import torch

from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, confusion_matrix

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, confusion_matrix
import numpy as np
import time

from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
import optuna

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## General AMP

### Data

In [37]:
dbbasp 

,ID,COMPLEXITY,NAME,N TERMINUS,SEQUENCE,C TERMINUS,SYNTHESIS TYPE,TARGET GROUP,TARGET OBJECT
0,1,Multimer,Distinctin,NaN,NLVSGLIEARKYLEQLHRKLKNCKV ENREVPPGFTALIKTLR...,NaN,Ribosomal,Gram+ Gram-,Lipid Bilayer
1,3,Multimer,Halocidin,NaN,WLNALLHHGLNCAKGVLA ALLHHGLNCAKGVLA,AMD AMD,Ribosomal,Gram+,Lipid Bilayer
2,4,Multimer,Khal,NaN,KWLNALLHHGLNCAKGVLA ALLHHGLNCAKGVLA,AMD AMD,Synthetic,Gram+ Gram-,Lipid Bilayer
3,6,Multi-Peptide,Enterocin X,NaN,SNDSLWYGVGQFMGKQANCITNHPVKHMIIPGYCLSKILG IA...,NaN,Ribosomal,Gram+,Lipid Bilayer
4,7,Multi-Peptide,EAFP1 + EAFP2,NaN,XTCASRCPRPCNAGLCCSIYGYCGSGNAYCGAGNCRCQCRG X...,NaN,Ribosomal,Fungus,Lipid Bilayer
...,...,...,...,...,...,...,...,...,...
23322,23586,Monomer,Litopeidin28-51,NaN,RRHRLIIRRRMRNSFWSSRACLAW,NaN,Synthetic,NaN,NaN
23324,23588,Monomer,NaN,NaN,RKIIRRIIPIIRRIIKR,NaN,Synthetic,NaN,NaN
23326,23590,Monomer,NaN,NaN,IRIIRRRRRIIRI,NaN,Synthetic,NaN,NaN
23327,23591,Monomer,NaN,NaN,RRIIRIHIRIIRR,NaN,Synthetic,NaN,NaN


In [38]:
dbbasp = pd.read_csv("../models1/database_check/DBAASP_peptides.csv")
dbbasp = dbbasp[dbbasp["SEQUENCE"].str.len() >= 10]
dbbasp = dbbasp[~dbbasp["TARGET GROUP"].str.contains("Fungus", na=False)]
dbbasp = dbbasp[["ID", "SEQUENCE"]]
dbbasp.columns = ["Peptide ID", "Sequence"]
adam_df = pd.read_csv("../data/naturalAMPs_APD2024a-ADAM.csv")
adam_df = pd.concat([adam_df, dbbasp], ignore_index=True)
# uniprot_df = pd.read_csv("../data/uniprotkb_length_10_TO_80_NOT_antimicro_2025_04_14.fasta.csv")
# uniprot_df1 = pd.read_csv("../data/uniprotkb_length_10_TO_80_NOT_antimicro_2025_04_14.fasta1.csv")
# uniprot_df = pd.concat([uniprot_df, uniprot_df1], ignore_index=True)

#### raw data gen

In [39]:
import torch
import pandas as pd
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Load only positive (AMP) sequences
adam_df = pd.read_csv("../data/naturalAMPs_APD2024a-ADAM.csv")

unique_letters = set(''.join(adam_df["Sequence"]))
print(unique_letters)
print(len(unique_letters))
print(f"Number of sequences after filtering: {len(adam_df)}")
adam_df = adam_df.drop_duplicates(subset='Sequence')
tb_df = pd.read_csv('../data/all_seq702.csv')
adam_df = adam_df[~adam_df['Sequence'].isin(tb_df['Sequences'])]
adam_df = adam_df[adam_df["Sequence"].str.len() >= 10]
generation_seqs = adam_df["Sequence"].reset_index(drop=True)

import torch
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence

def one_hot_torch(seq: str, dtype=torch.float32):
    amino_acids = "ACDEFGHIKLMNPQRSTVWY"
    seq_bytes = torch.ByteTensor(list(bytes(seq, "utf-8")))
    aa_bytes = torch.ByteTensor(list(bytes(amino_acids, "utf-8")))
    arr = torch.zeros(len(amino_acids), len(seq_bytes), dtype=dtype)
    for i, aa in enumerate(aa_bytes):
        arr[i, seq_bytes == aa] = 1
    return arr

class GenerativeSequenceDataset(Dataset):
    def __init__(self, sequences, one_hot_dtype=torch.float32):
        self.sequences = sequences
        self.one_hot_dtype = one_hot_dtype

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences.iloc[idx]
        input_seq = seq[:-1]  # all residues except the last
        target_seq = seq[1:]  # all residues except the first
        length = len(input_seq.replace("X", ""))  # unpadded length
        input_one_hot = one_hot_torch(input_seq, dtype=self.one_hot_dtype)
        target_one_hot = one_hot_torch(target_seq, dtype=self.one_hot_dtype)
        # target_indices = torch.tensor(["ACDEFGHIKLMNPQRSTVWY".index(res) for res in target_seq], dtype=torch.long)
        return input_one_hot, target_one_hot, length

def generative_collate_and_pack(batch):
    sequences, targets, lengths = zip(*batch)

    lengths = torch.tensor(lengths)
    sorted_indices = torch.argsort(lengths, descending=True)
    sequences = [sequences[i] for i in sorted_indices]
    targets = [targets[i] for i in sorted_indices]
    lengths = lengths[sorted_indices]

    sequences = [seq.T for seq in sequences]  # transpose to [seq_len, features]
    targets = [tgt.T for tgt in targets]      # transpose targets as well

    padded_seqs = pad_sequence(sequences, batch_first=False)
    padded_targets = pad_sequence(targets, batch_first=False)

    packed_input = pack_padded_sequence(padded_seqs, lengths.cpu(), batch_first=False)
    packed_target = pack_padded_sequence(padded_targets, lengths.cpu(), batch_first=False)

    return packed_input, packed_target, lengths


# Train/val/test split
train_seqs, test_seqs = train_test_split(generation_seqs, test_size=0.3, random_state=42)
val_seqs, test_seqs = train_test_split(test_seqs, test_size=0.5, random_state=42)

train_dataset = GenerativeSequenceDataset(train_seqs)
val_dataset = GenerativeSequenceDataset(val_seqs)
test_dataset = GenerativeSequenceDataset(test_seqs)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=generative_collate_and_pack)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, collate_fn=generative_collate_and_pack)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=generative_collate_and_pack)

# Dataset sizes
dataset_sizes = {
    "Train": len(train_dataset),
    "Validation": len(val_dataset),
    "Test": len(test_dataset)
}
print("Dataset sizes:", dataset_sizes)

for x, y, l in train_loader:
    print("Input shape:", x.data.shape)  # [L, B, 20]
    print("Target shape:", y.data.shape)  # [L, B, 20]
    print("Lengths:", y.batch_sizes)  # Lengths of sequences in the batch
    break

{'H', 'R', 'F', 'E', 'C', 'G', 'W', 'D', 'N', 'I', 'M', 'A', 'Q', 'L', 'V', 'P', 'K', 'S', 'T', 'Y'}
20
Number of sequences after filtering: 3306
Dataset sizes: {'Train': 2219, 'Validation': 475, 'Test': 476}
Input shape: torch.Size([1962, 20])
Target shape: torch.Size([1962, 20])
Lengths: tensor([64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 63, 61, 61, 61, 60, 58,
        54, 51, 48, 46, 46, 40, 36, 35, 34, 31, 29, 28, 25, 24, 20, 18, 18, 17,
        14, 13, 12, 11, 11, 11, 11, 11,  9,  7,  6,  6,  6,  6,  6,  6,  6,  5,
         4,  4,  3,  3,  3,  3,  3,  3,  3,  2,  2,  2,  2,  2,  2,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1])


#### sliding window data gen

In [4]:
# # Re-import required libraries after environment reset
# import torch
# import pandas as pd
# from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
# from torch.utils.data import Dataset, DataLoader
# from sklearn.model_selection import train_test_split

# # Load only positive (AMP) sequences
# adam_df = pd.read_csv("../data/naturalAMPs_APD2024a-ADAM.csv")

# # Clean non-standard amino acids
# unique_letters = set(''.join(adam_df["Sequence"]))
# amino_acids = "ACDEFGHIKLMNPQRSTVWY"
# # non_standard_amino_acids = set(unique_letters) - set(amino_acids)
# # adam_df = adam_df[~adam_df["Sequence"].str.contains('|'.join(non_standard_amino_acids))]

# # Apply sliding window to generate fragments
# def generate_fragments(sequences, window_size=15, stride=5):
#     fragments = []
#     for seq in sequences:
#         for start in range(0, len(seq) - window_size + 1, stride):
#             fragment = seq[start:start + window_size]
#             fragments.append(fragment)
#     return fragments

# generation_fragments = generate_fragments(adam_df["Sequence"].tolist())

# # Define one-hot encoding function
# def one_hot_torch(seq: str, dtype=torch.float32):
#     amino_acids = "ACDEFGHIKLMNPQRSTVWY"
#     seq_bytes = torch.ByteTensor(list(bytes(seq, "utf-8")))
#     aa_bytes = torch.ByteTensor(list(bytes(amino_acids, "utf-8")))
#     arr = torch.zeros(len(amino_acids), len(seq_bytes), dtype=dtype)
#     for i, aa in enumerate(aa_bytes):
#         arr[i, seq_bytes == aa] = 1
#     return arr

# # Dataset using one-hot encoding for generative modeling
# class AMPGenerationOneHotDataset(Dataset):
#     def __init__(self, sequences):
#         self.sequences = sequences

#     def __len__(self):
#         return len(self.sequences)

#     def __getitem__(self, idx):
#         seq = self.sequences[idx]
#         input_seq = seq[:-1]  # all residues except the last
#         target_seq = seq[1:]  # all residues except the first
#         input_one_hot = one_hot_torch(input_seq)  # shape: [20, seq_len - 1]
#         target_one_hot = one_hot_torch(target_seq)  # shape: [20, seq_len - 1]
#         length = input_one_hot.shape[1]
#         return input_one_hot, target_one_hot, length

# # Collate function for packing one-hot sequences
# def collate_and_pack_for_generation(batch):
#     sequences, targets, lengths = zip(*batch)
#     lengths = torch.tensor(lengths)

#     # Sort by length (required for packing)
#     sorted_indices = torch.argsort(lengths, descending=True)
#     sequences = [sequences[i] for i in sorted_indices]
#     targets = [targets[i] for i in sorted_indices]
#     lengths = lengths[sorted_indices]

#     # Transpose each to [L, 20] and pad
#     sequences = [seq.T for seq in sequences]  # from [20, L] to [L, 20]
#     targets = [tgt.T for tgt in targets]      # from [20, L] to [L, 20]

#     padded_seqs = pad_sequence(sequences, batch_first=False)  # [L, B, 20]
#     padded_targets = pad_sequence(targets, batch_first=False)  # [L, B, 20]

#     packed_input = pack_padded_sequence(padded_seqs, lengths.cpu(), batch_first=False)
#     packed_target = pack_padded_sequence(padded_targets, lengths.cpu(), batch_first=False)

#     return packed_input, packed_target, lengths

# # Train/val/test split
# train_seqs, test_seqs = train_test_split(generation_fragments, test_size=0.3, random_state=42)
# val_seqs, test_seqs = train_test_split(test_seqs, test_size=0.5, random_state=42)

# train_dataset = AMPGenerationOneHotDataset(train_seqs)
# val_dataset = AMPGenerationOneHotDataset(val_seqs)
# test_dataset = AMPGenerationOneHotDataset(test_seqs)

# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate_and_pack_for_generation)
# val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, collate_fn=collate_and_pack_for_generation)
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=collate_and_pack_for_generation)

# # Dataset sizes for verification
# dataset_sizes = {
#     "Train": len(train_dataset),
#     "Validation": len(val_dataset),
#     "Test": len(test_dataset)
# }
# print("Dataset sizes:", dataset_sizes)

# for x, y, l in train_loader:
#     print("Input shape:", x.data.shape)  # [L, B, 20]
#     print("Target shape:", y.data.shape)  # [L, B, 20]
#     print("Lengths:", y.batch_sizes)  # Lengths of sequences in the batch
#     break

### model

In [4]:
class GenerativeLSTM(nn.Module):
    def __init__(self, input_dim=20, hidden_dim=128, num_layers=1, dropout=0.3, output_dim=20):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, dropout=dropout if num_layers > 1 else 0)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, packed_input):
        packed_output, _ = self.lstm(packed_input)
        dropped = self.dropout(packed_output.data)
        logits = self.fc(dropped)
        return logits  # shape: [total_timesteps, 20]

### test

In [6]:
model = GenerativeLSTM()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer and loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

# Train and eval functions
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for packed_input, packed_target, _ in dataloader:
        inputs = packed_input.to(device)
        targets = torch.argmax(packed_target.data, dim=1).to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for packed_input, packed_target, _ in dataloader:
            inputs = packed_input.to(device)
            targets = torch.argmax(packed_target.data, dim=1).to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
    return total_loss / len(dataloader)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, criterion, device)
    val_loss = evaluate(model, val_loader, criterion, device)
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

# Final test evaluation
test_loss = evaluate(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f}")


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f5ab3d21ed0>>
Traceback (most recent call last):
  File "/mnt/storageG1/lwang/miniconda3/envs/new-ml/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


Epoch 1/10 - Train Loss: 2.9105 - Val Loss: 2.8513
Epoch 2/10 - Train Loss: 2.8507 - Val Loss: 2.8381


: 

: 

## Model

In [5]:
from torch.nn.utils.rnn import pad_packed_sequence


class GenerativeLSTM(nn.Module):
    def __init__(self, input_dim=20, hidden_dim=128, num_layers=1, dropout=0.3):
        super(GenerativeLSTM, self).__init__()
        self.lstm = nn.LSTM(
            input_dim, hidden_dim, num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        self.fc = nn.Linear(hidden_dim, input_dim)

    def forward(self, x):
        # Handle packed input
        if isinstance(x, torch.nn.utils.rnn.PackedSequence):
            packed_output, _ = self.lstm(x)
            unpacked_output, _ = pad_packed_sequence(packed_output, batch_first=True)
            return self.fc(unpacked_output)
        else:
            out, _ = self.lstm(x)
            return self.fc(out)



#### General AMP - train

In [6]:
# Re-import necessary packages after reset
import torch
import torch.nn as nn
import torch.optim as optim
import datetime
import optuna
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, roc_auc_score
from torch.utils.tensorboard import SummaryWriter


# Criterion

PAD_IDX = -100
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)


def compute_last_token_loss(output, target_seq, criterion):
    """
    Computes cross-entropy loss on the last time step of each sequence.
    
    Args:
        output: Tensor of shape [B, L, vocab_size]
        target_seq: Tensor of shape [B, L] containing target class indices
    
    Returns:
        loss: Scalar loss computed only on the last token of each sequence
    """
    # Get last time step for each sequence
    last_token_logits = output[:, -1, :]        # [B, vocab_size]
    last_token_targets = target_seq[:, -1, :]      # [B]
    last_token_targets = torch.argmax(last_token_targets, dim=-1)  #  now shape is [batch_size, seq_len]

    # print('last_token_logits',last_token_logits.shape)
    # print('last_token_targets',last_token_targets.shape)

    return criterion(last_token_logits, last_token_targets)

# Training function
def train_model_generation(model, train_loader, val_loader, num_epochs=10, lr=1e-3, weight_decay=1e-4,
                           device='cuda' if torch.cuda.is_available() else 'cpu', verbose=False, train=True):
    model.to(device)
    if not train:
        model.eval()

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    best_val_loss = float('inf')
    log_dir = f"runs-lstm-gen/AMP_LSTM_GEN_{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
    writer = SummaryWriter(log_dir=log_dir)

    for epoch in range(1, num_epochs + 1):
        model.train()
        epoch_loss = 0.0
        for input_seq, target_seq, _ in train_loader:
            input_seq = input_seq.to(device)
            target_seq = target_seq.to(device)

            optimizer.zero_grad()
            output = model(input_seq)
            # output = output.view(-1, output.shape[-1])
            # output = output.reshape(-1, output.shape[-1])      # [B*L, vocab]
            # output = torch.argmax(output, dim=-1)  # now shape is [batch_size, seq_len]

            
            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)
            # # target_seq = target_seq.view(-1)
            # target_seq = torch.argmax(target_seq, dim=-1)  #  now shape is [batch_size, seq_len]

            
            # print('target_shape before reshape',target_seq.shape)
            # target_seq = target_seq.reshape(-1)
            # print(f"Output shape: {output.shape}, Target shape: {target_seq.shape}")

            # loss = criterion(output, target_seq)
            loss = compute_last_token_loss(output, target_seq, criterion)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        avg_train_loss = epoch_loss / len(train_loader)
        val_loss, acc, auc = evaluate_model_generation(model, val_loader, criterion, device, verbose)

        writer.add_scalar('Loss/Train', avg_train_loss, epoch)
        writer.add_scalar('Loss/Validation', val_loss, epoch)
        writer.add_scalar('Accuracy/Validation', acc, epoch)
        writer.add_scalar('AUC/Validation', auc if auc != "undefined" else 0.0, epoch)

        if verbose:
            print(f"Epoch [{epoch}/{num_epochs}] - Train Loss: {avg_train_loss:.4f}, Val Loss: {val_loss:.4f}, Acc: {acc:.4f}, AUC: {auc}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            if train:
                torch.save(model.state_dict(), 'best_model_lstm_generator.pt')

    writer.close()
    return best_val_loss

# Evaluation function
def evaluate_model_generation(model, data_loader, criterion, device='cuda' if torch.cuda.is_available() else 'cpu', verbose=False):
    model.eval()
    all_labels = []
    all_preds = []
    total_loss = 0.0
    with torch.no_grad():
        for input_seq, target_seq, _ in data_loader:
            input_seq = input_seq.to(device)
            target_seq = target_seq.to(device)

            output = model(input_seq)
            # output = output.view(-1, output.shape[-1])
            # output = output.reshape(-1, output.shape[-1])  # [B*L, vocab]
            # output = torch.argmax(output, dim=-1)  # now shape is [batch_size, seq_len]

            # # target_seq = target_seq.view(-1)
            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)
            # # target_seq = target_seq.view(-1)
            # # target_seq = target_seq.reshape(-1)
            # # target_seq = target_seq.reshape(-1, target_seq.shape[-1])
            # target_seq = torch.argmax(target_seq, dim=-1)  #  now shape is [batch_size, seq_len]

            # assert output.size(0) == target_seq.size(0), f"Mismatch: {output.size(0)} vs {target_seq.size(0)}"

            # loss = criterion(output, target_seq)
            loss = compute_last_token_loss(output, target_seq, criterion)
            # print('loss done')
            total_loss += loss.item()
            
            
            preds = output[:, -1, :]        # shape: [B, vocab_size]
            preds = torch.argmax(preds, dim=1)  # shape: [B]

            targets = target_seq[:, -1, :]      # shape: [B, vocab_size]
            targets = torch.argmax(targets, dim=-1)  # shape: [B]
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    avg_loss = total_loss / len(data_loader)
    acc = accuracy_score(all_labels, all_preds)
    # try:
    #     auc = roc_auc_score(
    #         torch.nn.functional.one_hot(torch.tensor(all_labels), num_classes=20),
    #         torch.nn.functional.one_hot(torch.tensor(all_preds), num_classes=20),
    #         multi_class='ovr', average='macro'
    #     )
    # except:
    auc = "undefined"

    return avg_loss, acc, auc

# Objective for Optuna tuning
def objective_generation(trial):
    hidden_dim = trial.suggest_int("hidden_dim", 64, 256)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    lr = trial.suggest_float("lr", 1e-4, 1e-2)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3)

    model = GenerativeLSTM(hidden_dim=hidden_dim, num_layers=num_layers, dropout=dropout)
    val_loss = train_model_generation(model, train_loader, val_loader, num_epochs=10, lr=lr, weight_decay=weight_decay, verbose=False)
    return val_loss

study = optuna.create_study(direction="minimize")
study.optimize(objective_generation, n_trials=20)

lstm_gen_best_params = study.best_trial.params
print(lstm_gen_best_params)

[I 2025-04-24 09:42:23,365] A new study created in memory with name: no-name-39e4a52e-15b8-43ed-af6c-b28197acf9d7
[I 2025-04-24 09:43:19,587] Trial 0 finished with value: 0.29205354303121567 and parameters: {'hidden_dim': 170, 'num_layers': 2, 'dropout': 0.2656485138314876, 'lr': 0.008011877132915504, 'weight_decay': 8.31394692866913e-05}. Best is trial 0 with value: 0.29205354303121567.
[I 2025-04-24 09:44:07,686] Trial 1 finished with value: 0.543607734143734 and parameters: {'hidden_dim': 227, 'num_layers': 2, 'dropout': 0.42816344492915914, 'lr': 0.005571700887563167, 'weight_decay': 0.0006397652896710152}. Best is trial 0 with value: 0.29205354303121567.
[I 2025-04-24 09:44:48,540] Trial 2 finished with value: 1.1754698157310486 and parameters: {'hidden_dim': 195, 'num_layers': 2, 'dropout': 0.4354436052697497, 'lr': 0.003323673614997853, 'weight_decay': 0.0001282722171795435}. Best is trial 0 with value: 0.29205354303121567.
[I 2025-04-24 09:45:22,209] Trial 3 finished with value

{'hidden_dim': 247, 'num_layers': 3, 'dropout': 0.12915638774083413, 'lr': 0.009983517577195865, 'weight_decay': 0.0007901836295730717}


In [ ]:
!tensorboard --logdir runs-lstm-gen

#### test

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, roc_auc_score
import math

# --- Assumes you already have these from your previous steps ---
# lstm_gen_best_params
# train_loader, val_loader, test_loader
# GenerativeLSTM
# compute_last_token_loss

PAD_IDX = -100
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_final_model(model, train_loader, val_loader, num_epochs=20):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lstm_gen_best_params["lr"], weight_decay=lstm_gen_best_params["weight_decay"])
    writer = SummaryWriter(log_dir=f"runs-lstm-gen/AMPGen_LSTM_final")

    best_val_loss = float('inf')

    for epoch in range(1, num_epochs + 1):
        model.train()
        total_loss = 0.0

        for input_seq, target_seq, _ in train_loader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            optimizer.zero_grad()

            output = model(input_seq)
            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        val_loss, acc, auc, perp = evaluate_final_model(model, test_loader)

        writer.add_scalar('Loss/Train', avg_train_loss, epoch)
        writer.add_scalar('Loss/Validation', val_loss, epoch)
        writer.add_scalar('Accuracy/Validation', acc, epoch)
        writer.add_scalar('AUC/Validation', auc if auc != "undefined" else 0.0, epoch)

        print(f"Epoch {epoch}: Train Loss = {avg_train_loss:.4f} | Val Loss = {val_loss:.4f} | Acc = {acc:.4f} | AUC = {auc} | Perplexity = {perp:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            # torch.save(model.state_dict(), "best_model_lstm_generator.pt")

    writer.close()
    return model

def evaluate_final_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    total_loss = 0.0

    with torch.no_grad():
        for input_seq, target_seq, _ in data_loader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            output = model(input_seq)

            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            total_loss += loss.item()

            preds = output[:, -1, :]  # [B, vocab]
            preds = torch.argmax(preds, dim=1)
            targets = target_seq[:, -1, :]
            targets = torch.argmax(targets, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    # try:
    #     auc = roc_auc_score(
    #         torch.nn.functional.one_hot(torch.tensor(all_labels), num_classes=20),
    #         torch.nn.functional.one_hot(torch.tensor(all_preds), num_classes=20),
    #         multi_class='ovr', average='macro'
    #     )
    # except Exception:
    auc = "undefined"

    avg_loss = total_loss / len(data_loader)
    perplexity = math.exp(avg_loss)
    return avg_loss, acc, auc, perplexity

# --- Build and train final model using best parameters ---
# lstm_gen_best_params = {'hidden_dim': 133, 'num_layers': 2, 'dropout': 0.10063270147175422, 'lr': 0.003237280156212186, 'weight_decay': 2.2594437829479466e-05}

final_model = GenerativeLSTM(
    input_dim=20,
    hidden_dim=lstm_gen_best_params["hidden_dim"],
    num_layers=lstm_gen_best_params["num_layers"],
    dropout=lstm_gen_best_params["dropout"]
)

trained_model = train_final_model(final_model, train_loader, val_loader, num_epochs=20)

# --- Evaluate on test set ---
test_loss, test_acc, test_auc, perp = evaluate_final_model(trained_model, test_loader)
print(f"\n✅ Final Test Metrics:\nLoss = {test_loss:.4f}, Accuracy = {test_acc:.4f}, AUC = {test_auc}, Perplexity = {perp:.4f}")
torch.save(trained_model.state_dict(), 'best_model_lstm_generator_final.pt')


Epoch 1: Train Loss = 2.6719 | Val Loss = 2.3381 | Acc = 0.9877 | AUC = undefined | Perplexity = 10.3614
Epoch 2: Train Loss = 2.0466 | Val Loss = 1.7488 | Acc = 0.9877 | AUC = undefined | Perplexity = 5.7476
Epoch 3: Train Loss = 1.5022 | Val Loss = 1.2576 | Acc = 0.9877 | AUC = undefined | Perplexity = 3.5169
Epoch 4: Train Loss = 1.0747 | Val Loss = 0.8921 | Acc = 0.9877 | AUC = undefined | Perplexity = 2.4403
Epoch 5: Train Loss = 0.7665 | Val Loss = 0.6423 | Acc = 0.9877 | AUC = undefined | Perplexity = 1.9009
Epoch 6: Train Loss = 0.5685 | Val Loss = 0.4766 | Acc = 0.9877 | AUC = undefined | Perplexity = 1.6106
Epoch 7: Train Loss = 0.4256 | Val Loss = 0.3775 | Acc = 0.9836 | AUC = undefined | Perplexity = 1.4586
Epoch 8: Train Loss = 0.3397 | Val Loss = 0.2988 | Acc = 0.9877 | AUC = undefined | Perplexity = 1.3483
Epoch 9: Train Loss = 0.2768 | Val Loss = 0.2534 | Acc = 0.9877 | AUC = undefined | Perplexity = 1.2884
Epoch 10: Train Loss = 0.2338 | Val Loss = 0.2171 | Acc = 0.987

## tb amp

In [3]:
df = pd.read_csv('../data/all_seq702.csv')
df = df.drop_duplicates(subset='Sequences')

max_length = df['Sequences'].str.len().max()
print(max_length)
# df['Sequences'] = df['Sequences'].apply(lambda x: x.ljust(max_length, 'X'))

unique_letters = set(''.join(df["Sequences"]))
print(unique_letters)
print(len(unique_letters))
amino_acids = set("ACDEFGHIKLMNPQRSTVWY")
non_standard_amino_acids = unique_letters - amino_acids
print(non_standard_amino_acids)
b_count = df["Sequences"].str.count('B').sum()
print(f"Number of 'B' values: {b_count}")
# manually replaced one of the B with D and the other with N

X = df["Sequences"]
y = df["AMP"]

# df_filtered = df[
#     (df['Sequences'].str.len() >= 10) &
#     (df['Sequences'].apply(lambda x: len(set(x)) > 1)) &
#     (~df['Sequences'].str.contains('X'))
# ]
df_filtered = df

def split_sequence(seq, chunk_size=20):
    return [seq[i:i+chunk_size] for i in range(0, len(seq), chunk_size)]

new_rows = []
for _, row in df_filtered.iterrows():
    seq = row['Sequences']
    amp_label = row['AMP']
    if len(seq) > 40:
        for chunk in split_sequence(seq, 20):
            new_rows.append({'Sequences': chunk, 'AMP': amp_label})
    else:
        new_rows.append({'Sequences': seq, 'AMP': amp_label})

df_filtered = pd.DataFrame(new_rows)


df_filtered = df_filtered[
    (df_filtered['Sequences'].str.len() >= 10) &
    (df_filtered['Sequences'].apply(lambda x: len(set(x)) > 1)) &
    (~df_filtered['Sequences'].str.contains('X'))
]
df_filtered = df_filtered[df_filtered['AMP']==1]
df_filtered = df_filtered.drop_duplicates(subset='Sequences')


128
{'H', 'R', 'F', 'E', 'C', 'G', 'W', 'D', 'N', 'I', 'M', 'A', 'Q', 'L', 'P', 'V', 'K', 'X', 'S', 'T', 'Y'}
21
{'X'}
Number of 'B' values: 0


In [4]:
print(min([len(x) for x in df_filtered['Sequences']]))
print(max([len(x) for x in df_filtered['Sequences']]))

10
40


In [5]:
# Re-import libraries after environment reset
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
from sklearn.model_selection import train_test_split

# Load data
df = df_filtered

# Clean and inspect
amino_acids = set("ACDEFGHIKLMNPQRSTVWY")
unique_letters = set(''.join(df["Sequences"]))
# non_standard_amino_acids = unique_letters - amino_acids
# df = df[~df["Sequences"].str.contains('|'.join(non_standard_amino_acids))]

# Extract sequences
sequences = df["Sequences"].reset_index(drop=True)

# Define one-hot function
def one_hot_torch(seq: str, dtype=torch.float32):
    amino_acids = "ACDEFGHIKLMNPQRSTVWY"
    seq_bytes = torch.ByteTensor(list(bytes(seq, "utf-8")))
    aa_bytes = torch.ByteTensor(list(bytes(amino_acids, "utf-8")))
    arr = torch.zeros(len(amino_acids), len(seq_bytes), dtype=dtype)
    for i, aa in enumerate(aa_bytes):
        arr[i, seq_bytes == aa] = 1
    return arr

# Define dataset class
class GenerativeSequenceDataset(Dataset):
    def __init__(self, sequences, one_hot_dtype=torch.float32):
        self.sequences = sequences
        self.one_hot_dtype = one_hot_dtype

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences.iloc[idx]
        input_seq = seq[:-1]
        target_seq = seq[1:]
        length = len(input_seq.replace("X", ""))
        input_one_hot = one_hot_torch(input_seq, dtype=self.one_hot_dtype)
        target_one_hot = one_hot_torch(target_seq, dtype=self.one_hot_dtype)
        return input_one_hot, target_one_hot, length

# Define collate function
def generative_collate_and_pack(batch):
    sequences, targets, lengths = zip(*batch)
    lengths = torch.tensor(lengths)
    sorted_indices = torch.argsort(lengths, descending=True)
    sequences = [sequences[i] for i in sorted_indices]
    targets = [targets[i] for i in sorted_indices]
    lengths = lengths[sorted_indices]
    sequences = [seq.T for seq in sequences]
    targets = [tgt.T for tgt in targets]
    padded_seqs = pad_sequence(sequences, batch_first=False)
    padded_targets = pad_sequence(targets, batch_first=False)
    packed_input = pack_padded_sequence(padded_seqs, lengths.cpu(), batch_first=False)
    packed_target = pack_padded_sequence(padded_targets, lengths.cpu(), batch_first=False)
    return packed_input, packed_target, lengths

# Split and load data
train_seqs, test_seqs = train_test_split(sequences, test_size=0.3, random_state=42)
val_seqs, test_seqs = train_test_split(test_seqs, test_size=0.5, random_state=42)
train_dataset = GenerativeSequenceDataset(train_seqs)
val_dataset = GenerativeSequenceDataset(val_seqs)
test_dataset = GenerativeSequenceDataset(test_seqs)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=generative_collate_and_pack)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, collate_fn=generative_collate_and_pack)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=generative_collate_and_pack)

# Preview batch
batch_sample = next(iter(train_loader))
batch_sample_shapes = {
    "Input shape": batch_sample[0].data.shape,
    "Target shape": batch_sample[1].data.shape,
    "Lengths": batch_sample[0].batch_sizes
}
batch_sample_shapes


{'Input shape': torch.Size([1199, 20]),
 'Target shape': torch.Size([1199, 20]),
 'Lengths': tensor([64, 64, 64, 64, 64, 64, 64, 64, 64, 57, 53, 47, 44, 43, 41, 39, 37, 36,
         34, 21, 21, 20, 18, 17, 13,  8,  8,  8,  8,  8,  8,  8,  7,  5,  4,  4,
          3,  3])}

In [6]:
dataset_sizes = {
    "Train": len(train_dataset),
    "Validation": len(val_dataset),
    "Test": len(test_dataset)
}
print("Dataset sizes:")
for name, size in dataset_sizes.items():
    print(f"{name}: {size}")

Dataset sizes:
Train: 171
Validation: 37
Test: 37


### train for full backprop

In [12]:

# Criterion
PAD_IDX = -100
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

# Transfer Learning Loader
def load_pretrained_weights(model, checkpoint_path):
    pretrained_dict = torch.load(checkpoint_path, map_location='cpu')
    model.load_state_dict(pretrained_dict, strict=False)
    return model

def compute_last_token_loss(output, target_seq, criterion):
    last_token_logits = output[:, -1, :]
    last_token_targets = target_seq[:, -1, :]
    last_token_targets = torch.argmax(last_token_targets, dim=-1)
    return criterion(last_token_logits, last_token_targets)

# Training function
def train_model_generation(model, train_loader, val_loader, num_epochs=10, lr=1e-3, weight_decay=1e-4,
                           device='cuda' if torch.cuda.is_available() else 'cpu', verbose=False, train=True):
    model.to(device)
    if not train:
        model.eval()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    best_val_loss = float('inf')
    log_dir = f"runs-lstm-gen-tb/AMP_LSTM_GEN_TRANSFER_{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
    writer = SummaryWriter(log_dir=log_dir)

    for epoch in range(1, num_epochs + 1):
        model.train()
        epoch_loss = 0.0
        for input_seq, target_seq, _ in train_loader:
            input_seq = input_seq.to(device)
            target_seq = target_seq.to(device)

            optimizer.zero_grad()
            output = model(input_seq)

            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        avg_train_loss = epoch_loss / len(train_loader)
        val_loss, acc, auc = evaluate_model_generation(model, val_loader, criterion, device, verbose)

        writer.add_scalar('Loss/Train', avg_train_loss, epoch)
        writer.add_scalar('Loss/Validation', val_loss, epoch)
        writer.add_scalar('Accuracy/Validation', acc, epoch)
        writer.add_scalar('AUC/Validation', auc if auc != "undefined" else 0.0, epoch)

        if verbose:
            print(f"Epoch [{epoch}/{num_epochs}] - Train Loss: {avg_train_loss:.4f}, Val Loss: {val_loss:.4f}, Acc: {acc:.4f}, AUC: {auc}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            # if train:
            #     torch.save(model.state_dict(), 'best_model_lstm_transfer.pt')

    writer.close()
    return best_val_loss

# Evaluation function
def evaluate_model_generation(model, data_loader, criterion, device='cuda' if torch.cuda.is_available() else 'cpu', verbose=False):
    model.eval()
    all_labels = []
    all_preds = []
    total_loss = 0.0
    with torch.no_grad():
        for input_seq, target_seq, _ in data_loader:
            input_seq = input_seq.to(device)
            target_seq = target_seq.to(device)

            output = model(input_seq)

            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            total_loss += loss.item()

            preds = output[:, -1, :]
            preds = torch.argmax(preds, dim=1)

            targets = target_seq[:, -1, :]
            targets = torch.argmax(targets, dim=-1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    avg_loss = total_loss / len(data_loader)
    acc = accuracy_score(all_labels, all_preds)
    # try:
    #     auc = roc_auc_score(
    #         torch.nn.functional.one_hot(torch.tensor(all_labels), num_classes=20),
    #         torch.nn.functional.one_hot(torch.tensor(all_preds), num_classes=20),
    #         multi_class='ovr', average='macro'
    #     )
    # except:
    auc = "undefined"
    
    return avg_loss, acc, auc

# Optuna objective for fine-tuning
def objective_generation(trial):
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    lr = trial.suggest_float("lr", 1e-4, 1e-2)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3)

    model = GenerativeLSTM(    
                hidden_dim=lstm_gen_best_params["hidden_dim"],
                num_layers=lstm_gen_best_params["num_layers"],
                dropout=dropout
                # dropout=lstm_gen_best_params["dropout"]
                )
    model = load_pretrained_weights(model, 'best_model_lstm_generator_final.pt')  # path to the general AMP model
    val_loss = train_model_generation(model, train_loader, val_loader, num_epochs=20, lr=lr, weight_decay=weight_decay, verbose=False)
    return val_loss

# Run Optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective_generation, n_trials=20)
lstm_gen_best_params_tb = study.best_trial.params
print("Best transfer learning hyperparameters:", lstm_gen_best_params_tb)

[I 2025-04-24 10:11:14,059] A new study created in memory with name: no-name-e9ca107c-4e23-4873-85ca-c69bf279a146
[I 2025-04-24 10:11:19,896] Trial 0 finished with value: 0.18188254535198212 and parameters: {'dropout': 0.24678797353839577, 'lr': 0.008594761299464238, 'weight_decay': 0.0009517778012328149}. Best is trial 0 with value: 0.18188254535198212.
[I 2025-04-24 10:11:25,828] Trial 1 finished with value: 0.2015029788017273 and parameters: {'dropout': 0.3909797957559561, 'lr': 0.006216064073374344, 'weight_decay': 0.000784507995534682}. Best is trial 0 with value: 0.18188254535198212.
[I 2025-04-24 10:11:31,313] Trial 2 finished with value: 0.20068062841892242 and parameters: {'dropout': 0.12298074582499506, 'lr': 0.00011367954424524167, 'weight_decay': 0.000785148468503358}. Best is trial 0 with value: 0.18188254535198212.
[I 2025-04-24 10:11:37,014] Trial 3 finished with value: 0.19988277554512024 and parameters: {'dropout': 0.3417578994521054, 'lr': 0.007401726162839175, 'weigh

Best transfer learning hyperparameters: {'dropout': 0.13882938162931305, 'lr': 0.001464264209101335, 'weight_decay': 0.0002912839503135125}


#### test

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, roc_auc_score
import math

# --- Assumes you already have these from your previous steps ---
# lstm_gen_best_params
# train_loader, val_loader, test_loader
# GenerativeLSTM
# compute_last_token_loss

PAD_IDX = -100
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_final_model(model, train_loader, val_loader, num_epochs=20):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lstm_gen_best_params_tb["lr"], weight_decay=lstm_gen_best_params_tb["weight_decay"])
    writer = SummaryWriter(log_dir=f"runs-lstm-gen-tb/AMPGen_LSTM_final")

    best_val_loss = float('inf')

    for epoch in range(1, num_epochs + 1):
        model.train()
        total_loss = 0.0

        for input_seq, target_seq, _ in train_loader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            optimizer.zero_grad()

            output = model(input_seq)
            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        val_loss, acc, auc, perp = evaluate_final_model(model, test_loader)

        writer.add_scalar('Loss/Train', avg_train_loss, epoch)
        writer.add_scalar('Loss/Validation', val_loss, epoch)
        writer.add_scalar('Accuracy/Validation', acc, epoch)
        writer.add_scalar('AUC/Validation', auc if auc != "undefined" else 0.0, epoch)

        print(f"Epoch {epoch}: Train Loss = {avg_train_loss:.4f} | Val Loss = {val_loss:.4f} | Acc = {acc:.4f} | AUC = {auc} | Perplexity = {perp:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            # torch.save(model.state_dict(), "best_model_lstm_generator.pt")

    writer.close()
    return model

def evaluate_final_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    total_loss = 0.0

    with torch.no_grad():
        for input_seq, target_seq, _ in data_loader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            output = model(input_seq)

            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            total_loss += loss.item()

            preds = output[:, -1, :]  # [B, vocab]
            preds = torch.argmax(preds, dim=1)
            targets = target_seq[:, -1, :]
            targets = torch.argmax(targets, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    # try:
    #     auc = roc_auc_score(
    #         torch.nn.functional.one_hot(torch.tensor(all_labels), num_classes=20),
    #         torch.nn.functional.one_hot(torch.tensor(all_preds), num_classes=20),
    #         multi_class='ovr', average='macro'
    #     )
    # except Exception:
    auc = "undefined"

    avg_loss = total_loss / len(data_loader)
    perplexity = math.exp(avg_loss)
    return avg_loss, acc, auc, perplexity

# --- Build and train final model using best parameters ---

# lstm_gen_best_params = {'hidden_dim': 133, 'num_layers': 2}

final_model = GenerativeLSTM(
    input_dim=20,
    hidden_dim=lstm_gen_best_params["hidden_dim"],
    num_layers=lstm_gen_best_params["num_layers"],
    dropout=lstm_gen_best_params_tb["dropout"],
    # weights_decay=lstm_gen_best_params_tb["weight_decay"],
    # lr=lstm_gen_best_params_tb["lr"]
)

trained_model = train_final_model(final_model, train_loader, val_loader, num_epochs=200)

# --- Evaluate on test set ---
test_loss, test_acc, test_auc, perp = evaluate_final_model(trained_model, test_loader)
print(f" Final Test Metrics:\nLoss = {test_loss:.4f}, Accuracy = {test_acc:.4f}, AUC = {test_auc}, Perplexity = {perp:.4f}")
# Save model weights
torch.save(trained_model.state_dict(), "final_amp_generator_lstm.pt")
print("Model saved to final_amp_generator_lstm.pt")


Epoch 1: Train Loss = 3.0329 | Val Loss = 3.0233 | Acc = 0.0270 | AUC = undefined | Perplexity = 20.5580
Epoch 2: Train Loss = 3.0231 | Val Loss = 3.0125 | Acc = 0.0270 | AUC = undefined | Perplexity = 20.3385
Epoch 3: Train Loss = 3.0123 | Val Loss = 2.9993 | Acc = 0.0270 | AUC = undefined | Perplexity = 20.0712
Epoch 4: Train Loss = 2.9970 | Val Loss = 2.9754 | Acc = 0.0270 | AUC = undefined | Perplexity = 19.5973
Epoch 5: Train Loss = 2.9844 | Val Loss = 2.9430 | Acc = 0.0270 | AUC = undefined | Perplexity = 18.9727
Epoch 6: Train Loss = 2.9729 | Val Loss = 2.9306 | Acc = 0.0270 | AUC = undefined | Perplexity = 18.7394
Epoch 7: Train Loss = 2.9773 | Val Loss = 2.9341 | Acc = 0.0000 | AUC = undefined | Perplexity = 18.8044
Epoch 8: Train Loss = 2.9475 | Val Loss = 2.9320 | Acc = 0.0000 | AUC = undefined | Perplexity = 18.7655
Epoch 9: Train Loss = 2.9300 | Val Loss = 2.9232 | Acc = 0.0000 | AUC = undefined | Perplexity = 18.6015
Epoch 10: Train Loss = 2.9664 | Val Loss = 2.9157 | Acc

#### Generation

In [14]:
import torch
import torch.nn.functional as F
import numpy as np

# Recreate your amino acid vocab
aa_vocab = 'ACDEFGHIKLMNPQRSTVWY'
aa_to_idx = {aa: i for i, aa in enumerate(aa_vocab)}
idx_to_aa = {i: aa for aa, i in aa_to_idx.items()}

def one_hot_encode_amino_acid(aa, vocab=aa_vocab):
    vec = torch.zeros(len(vocab))
    vec[aa_to_idx[aa]] = 1.0
    return vec

def generate_sequence_from_seed(model, seed, max_length=30, temperature=1.0, device='cpu'):
    model.eval()
    input_seq = [one_hot_encode_amino_acid(aa).to(device) for aa in seed]
    input_tensor = torch.stack(input_seq).unsqueeze(0)  # [1, L, 20]

    generated = seed.copy()

    with torch.no_grad():
        for _ in range(max_length - len(seed)):
            output = model(input_tensor)  # [1, L, vocab]
            logits = output[0, -1, :]  # Last time step → [vocab]

            # Apply temperature and sample
            probs = F.softmax(logits / temperature, dim=-1).cpu().numpy()
            next_idx = np.random.choice(len(aa_vocab), p=probs)
            next_aa = idx_to_aa[next_idx]

            # Update sequence
            next_aa_vec = one_hot_encode_amino_acid(next_aa).to(device).unsqueeze(0).unsqueeze(0)  # [1, 1, 20]
            input_tensor = torch.cat([input_tensor, next_aa_vec], dim=1)
            generated.append(next_aa)

    return ''.join(generated)

import numpy as np
from collections import Counter

class LengthSampler:
    def __init__(self, sequence_lengths):
        """
        Initialize sampler from observed sequence lengths.
        
        Args:
            sequence_lengths (list[int]): List of sequence lengths (e.g., [20, 21, 20, 23, ...])
        """
        self.length_counts = Counter(sequence_lengths)
        self.lengths = np.array(sorted(self.length_counts.keys()))
        counts = np.array([self.length_counts[l] for l in self.lengths])
        self.probs = counts / counts.sum()  # Empirical probabilities

    def sample(self, n=1):
        """
        Sample one or more lengths based on the learned distribution.

        Returns:
            np.ndarray of sampled lengths
        """
        return np.random.choice(self.lengths, size=n, p=self.probs)
length_sampler = LengthSampler([len(seq) for seq in df.loc[df['AMP'] == 1, :]['Sequences']])

In [15]:
import random
import numpy as np

# Set global seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Define the amino acid vocabulary
amino_acids = "ACDEFGHIKLMNPQRSTVWY"

def sample_start_amino_acid():
    return random.choice(amino_acids)

# Reload the trained model
gen_model = GenerativeLSTM(
    input_dim=20,
    hidden_dim=lstm_gen_best_params["hidden_dim"],
    num_layers=lstm_gen_best_params["num_layers"],
    # dropout=lstm_gen_best_params["dropout"]
)
gen_model.load_state_dict(torch.load("final_amp_generator_lstm.pt"))
gen_model.to(device)
gen_model.eval()
# Define a seed and generate a sequence
sampled_length = length_sampler.sample()[0]
start_aa = sample_start_amino_acid()
seed_sequence = list(start_aa)  # start with valine
generated_peptide = generate_sequence_from_seed(gen_model, seed_sequence, max_length=sampled_length, temperature=1.0, device=device)

print("Generated AMP sequence:", generated_peptide)

Generated AMP sequence: EWQMDDAVNPAYRGGG


#### batch gen

In [16]:
import random
import numpy as np

# Set global seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Define the amino acid vocabulary
amino_acids = "ACDEFGHIKLMNPQRSTVWY"

def sample_start_amino_acid():
    return random.choice(amino_acids)

# Reload the trained model
gen_model = GenerativeLSTM(
    input_dim=20,
    hidden_dim=lstm_gen_best_params["hidden_dim"],
    num_layers=lstm_gen_best_params["num_layers"],
    # dropout=lstm_gen_best_params["dropout"]
)
gen_model.load_state_dict(torch.load("final_amp_generator_lstm.pt"))
gen_model.to(device)
gen_model.eval()
generated_peptides = []
# (Re-run the generation loop to collect sequences)
for x in range(100):
    sampled_length = length_sampler.sample()[0]
    # sampled_length = 20
    start_aa = sample_start_amino_acid()
    seed_sequence = list(start_aa)
    generated_peptide = generate_sequence_from_seed(gen_model, seed_sequence, max_length=sampled_length, temperature=1.2, device=device)
    generated_peptides.append(generated_peptide)
    print("Generated AMP sequence:", generated_peptide)

# Save all generated sequences into a text file
with open("generated_peptides.fasta", "w") as f:
    for i, peptide in enumerate(generated_peptides):
        f.write(f">peptide{i}\n")
        f.write(peptide + "\n")

Generated AMP sequence: EWQNDDAVNPAYSGGG
Generated AMP sequence: ALIGNDGHKRGKLD
Generated AMP sequence: KCAYYSGDPKFKDYHPHKKG
Generated AMP sequence: IRWVNWCGAHHHSHHKGPGYPGDPPPPGHGTLHGHHPPY
Generated AMP sequence: IAQRMRKKIAECPHKYGHP
Generated AMP sequence: FAFDWSPVSGYK
Generated AMP sequence: EVGAFISTAKHGGHYHKPHYYGKHHF
Generated AMP sequence: VKAFWGDKYGPPGPHPPKGR
Generated AMP sequence: DCAMPAKGPGPHYG
Generated AMP sequence: WAWVFPSLKGDYYP
Generated AMP sequence: QGQVVRPCFYNAGP
Generated AMP sequence: CCLPPFQGHQ
Generated AMP sequence: ATPMAHGGYHYPRKKKGPHD
Generated AMP sequence: DCWYWHAYIYYTHHSHGKYP
Generated AMP sequence: HANYCKVQPPHHRRWYKKPP
Generated AMP sequence: ISVGHCMAKKHMDDRHGKPG
Generated AMP sequence: TAALLPQYKHRHIGDYRPHG
Generated AMP sequence: YELQPGYQLPHG
Generated AMP sequence: ARAAAATPKDKKGHH
Generated AMP sequence: VNAHPKTPFDPCLYKHPHKY
Generated AMP sequence: HYVDACADPDHSCRHG
Generated AMP sequence: VNVQSGERSYHHQHYTHPPG
Generated AMP sequence: QKSGVHAWDHYYKPIHHPPPPGKGK

### frozen

In [19]:

# Criterion
PAD_IDX = -100
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

# Transfer Learning Loader
def load_pretrained_weights(model, checkpoint_path):
    pretrained_dict = torch.load(checkpoint_path, map_location='cpu')
    model.load_state_dict(pretrained_dict, strict=False)
    return model

def freeze_encoder(model):
    for name, param in model.named_parameters():
        if 'lstm' in name:
            param.requires_grad = False

def compute_last_token_loss(output, target_seq, criterion):
    last_token_logits = output[:, -1, :]
    last_token_targets = target_seq[:, -1, :]
    last_token_targets = torch.argmax(last_token_targets, dim=-1)
    return criterion(last_token_logits, last_token_targets)

# Training function
def train_model_generation(model, train_loader, val_loader, num_epochs=10, lr=1e-3, weight_decay=1e-4,
                           device='cuda' if torch.cuda.is_available() else 'cpu', verbose=False, train=True):
    model.to(device)
    if not train:
        model.eval()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    best_val_loss = float('inf')
    log_dir = f"runs-lstm-gen-tb/AMP_LSTM_GEN_TRANSFER_{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
    writer = SummaryWriter(log_dir=log_dir)

    for epoch in range(1, num_epochs + 1):
        model.train()
        epoch_loss = 0.0
        for input_seq, target_seq, _ in train_loader:
            input_seq = input_seq.to(device)
            target_seq = target_seq.to(device)

            optimizer.zero_grad()
            output = model(input_seq)

            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        avg_train_loss = epoch_loss / len(train_loader)
        val_loss, acc, auc = evaluate_model_generation(model, val_loader, criterion, device, verbose)

        writer.add_scalar('Loss/Train', avg_train_loss, epoch)
        writer.add_scalar('Loss/Validation', val_loss, epoch)
        writer.add_scalar('Accuracy/Validation', acc, epoch)
        writer.add_scalar('AUC/Validation', auc if auc != "undefined" else 0.0, epoch)

        if verbose:
            print(f"Epoch [{epoch}/{num_epochs}] - Train Loss: {avg_train_loss:.4f}, Val Loss: {val_loss:.4f}, Acc: {acc:.4f}, AUC: {auc}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            # if train:
            #     torch.save(model.state_dict(), 'best_model_lstm_transfer.pt')

    writer.close()
    return best_val_loss

# Evaluation function
def evaluate_model_generation(model, data_loader, criterion, device='cuda' if torch.cuda.is_available() else 'cpu', verbose=False):
    model.eval()
    all_labels = []
    all_preds = []
    total_loss = 0.0
    with torch.no_grad():
        for input_seq, target_seq, _ in data_loader:
            input_seq = input_seq.to(device)
            target_seq = target_seq.to(device)

            output = model(input_seq)

            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            total_loss += loss.item()

            preds = output[:, -1, :]
            preds = torch.argmax(preds, dim=1)

            targets = target_seq[:, -1, :]
            targets = torch.argmax(targets, dim=-1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    avg_loss = total_loss / len(data_loader)
    acc = accuracy_score(all_labels, all_preds)
    # try:
    #     auc = roc_auc_score(
    #         torch.nn.functional.one_hot(torch.tensor(all_labels), num_classes=20),
    #         torch.nn.functional.one_hot(torch.tensor(all_preds), num_classes=20),
    #         multi_class='ovr', average='macro'
    #     )
    # except:
    auc = "undefined"
    
    return avg_loss, acc, auc

# Optuna objective for fine-tuning
def objective_generation(trial):
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    lr = trial.suggest_float("lr", 1e-4, 1e-2)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3)

    model = GenerativeLSTM(    
                hidden_dim=lstm_gen_best_params["hidden_dim"],
                num_layers=lstm_gen_best_params["num_layers"],
                dropout=dropout
                # dropout=lstm_gen_best_params["dropout"]
                )
    model = load_pretrained_weights(model, 'best_model_lstm_generator_final.pt')  # path to the general AMP model
    freeze_encoder(model)

    val_loss = train_model_generation(model, train_loader, val_loader, num_epochs=20, lr=lr, weight_decay=weight_decay, verbose=False)
    return val_loss

# Run Optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective_generation, n_trials=20)
lstm_gen_frozen_best_params_tb = study.best_trial.params
print("Best transfer learning hyperparameters:", lstm_gen_frozen_best_params_tb)

[I 2025-04-24 10:28:25,991] A new study created in memory with name: no-name-b18246ca-b8ce-44e6-8d69-a246afdf7863
[I 2025-04-24 10:28:30,732] Trial 0 finished with value: 0.19571612775325775 and parameters: {'dropout': 0.3738698734696153, 'lr': 0.008929602955292291, 'weight_decay': 0.00038939709500130466}. Best is trial 0 with value: 0.19571612775325775.
[I 2025-04-24 10:28:35,480] Trial 1 finished with value: 0.19314280152320862 and parameters: {'dropout': 0.41696652333028095, 'lr': 0.003128895078184863, 'weight_decay': 0.0006020832987418363}. Best is trial 1 with value: 0.19314280152320862.
[I 2025-04-24 10:28:40,667] Trial 2 finished with value: 0.20010095834732056 and parameters: {'dropout': 0.3497662374507716, 'lr': 0.0034644974046034883, 'weight_decay': 0.000795963699494901}. Best is trial 1 with value: 0.19314280152320862.
[I 2025-04-24 10:28:45,413] Trial 3 finished with value: 0.1991792917251587 and parameters: {'dropout': 0.18578045936297793, 'lr': 0.007855076852649084, 'weig

Best transfer learning hyperparameters: {'dropout': 0.2787933989664577, 'lr': 0.007081024979360626, 'weight_decay': 0.000648528973755982}


In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, roc_auc_score
import math

# --- Assumes you already have these from your previous steps ---
# lstm_gen_best_params
# train_loader, val_loader, test_loader
# GenerativeLSTM
# compute_last_token_loss

PAD_IDX = -100
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_final_model(model, train_loader, val_loader, num_epochs=20):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lstm_gen_frozen_best_params_tb["lr"], weight_decay=lstm_gen_frozen_best_params_tb["weight_decay"])
    writer = SummaryWriter(log_dir=f"runs-lstm-frozen-gen-tb/AMPGen_LSTM_final")

    best_val_loss = float('inf')

    for epoch in range(1, num_epochs + 1):
        model.train()
        total_loss = 0.0

        for input_seq, target_seq, _ in train_loader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            optimizer.zero_grad()

            output = model(input_seq)
            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        val_loss, acc, auc, perp = evaluate_final_model(model, test_loader)

        writer.add_scalar('Loss/Train', avg_train_loss, epoch)
        writer.add_scalar('Loss/Validation', val_loss, epoch)
        writer.add_scalar('Accuracy/Validation', acc, epoch)
        writer.add_scalar('AUC/Validation', auc if auc != "undefined" else 0.0, epoch)

        print(f"Epoch {epoch}: Train Loss = {avg_train_loss:.4f} | Val Loss = {val_loss:.4f} | Acc = {acc:.4f} | AUC = {auc} | Perplexity = {perp:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            # torch.save(model.state_dict(), "best_model_lstm_generator.pt")

    writer.close()
    return model

def evaluate_final_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    total_loss = 0.0

    with torch.no_grad():
        for input_seq, target_seq, _ in data_loader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            output = model(input_seq)

            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            total_loss += loss.item()

            preds = output[:, -1, :]  # [B, vocab]
            preds = torch.argmax(preds, dim=1)
            targets = target_seq[:, -1, :]
            targets = torch.argmax(targets, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    # try:
    #     auc = roc_auc_score(
    #         torch.nn.functional.one_hot(torch.tensor(all_labels), num_classes=20),
    #         torch.nn.functional.one_hot(torch.tensor(all_preds), num_classes=20),
    #         multi_class='ovr', average='macro'
    #     )
    # except Exception:
    auc = "undefined"

    avg_loss = total_loss / len(data_loader)
    perplexity = math.exp(avg_loss)
    return avg_loss, acc, auc, perplexity

# --- Build and train final model using best parameters ---

# lstm_gen_best_params = {'hidden_dim': 133, 'num_layers': 2, 'dropout': 0.10063270147175422, 'lr': 0.003237280156212186, 'weight_decay': 2.2594437829479466e-05}

final_model = GenerativeLSTM(
    input_dim=20,
    hidden_dim=lstm_gen_best_params["hidden_dim"],
    num_layers=lstm_gen_best_params["num_layers"],
    dropout=lstm_gen_frozen_best_params_tb["dropout"],
    # weights_decay=lstm_gen_frozen_best_params_tb["weight_decay"],
    # lr=lstm_gen_frozen_best_params_tb["lr"]
)
freeze_encoder(final_model)

trained_model = train_final_model(final_model, train_loader, val_loader, num_epochs=60)

# --- Evaluate on test set ---
test_loss, test_acc, test_auc, perp = evaluate_final_model(trained_model, test_loader)
print(f" Final Test Metrics:\nLoss = {test_loss:.4f}, Accuracy = {test_acc:.4f}, AUC = {test_auc}, Perplexity = {perp:.4f}")
# Save model weights
torch.save(trained_model.state_dict(), "final_amp_frozen_generator_lstm.pt")
print("Model saved to final_amp_frozen_generator_lstm.pt")


Epoch 1: Train Loss = 2.9309 | Val Loss = 2.9048 | Acc = 0.9730 | AUC = undefined | Perplexity = 18.2613
Epoch 2: Train Loss = 2.8908 | Val Loss = 2.8630 | Acc = 0.9730 | AUC = undefined | Perplexity = 17.5148
Epoch 3: Train Loss = 2.8480 | Val Loss = 2.8220 | Acc = 0.9730 | AUC = undefined | Perplexity = 16.8105
Epoch 4: Train Loss = 2.8080 | Val Loss = 2.7816 | Acc = 0.9730 | AUC = undefined | Perplexity = 16.1445
Epoch 5: Train Loss = 2.7681 | Val Loss = 2.7414 | Acc = 0.9730 | AUC = undefined | Perplexity = 15.5082
Epoch 6: Train Loss = 2.7323 | Val Loss = 2.7016 | Acc = 0.9730 | AUC = undefined | Perplexity = 14.9036
Epoch 7: Train Loss = 2.6880 | Val Loss = 2.6622 | Acc = 0.9730 | AUC = undefined | Perplexity = 14.3281
Epoch 8: Train Loss = 2.6532 | Val Loss = 2.6232 | Acc = 0.9730 | AUC = undefined | Perplexity = 13.7791
Epoch 9: Train Loss = 2.6103 | Val Loss = 2.5844 | Acc = 0.9730 | AUC = undefined | Perplexity = 13.2557
Epoch 10: Train Loss = 2.5796 | Val Loss = 2.5460 | Acc

In [27]:
import random
import numpy as np

# Set global seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Define the amino acid vocabulary
amino_acids = "ACDEFGHIKLMNPQRSTVWY"

def sample_start_amino_acid():
    return random.choice(amino_acids)

# Reload the trained model
gen_model = GenerativeLSTM(
    input_dim=20,
    hidden_dim=lstm_gen_best_params["hidden_dim"],
    num_layers=lstm_gen_best_params["num_layers"],
    dropout=lstm_gen_best_params["dropout"]
)
gen_model.load_state_dict(torch.load("final_amp_generator_lstm.pt"))
gen_model.to(device)

generated_peptides = []
# (Re-run the generation loop to collect sequences)
for x in range(100):
    sampled_length = length_sampler.sample()[0]
    # sampled_length = 20
    start_aa = sample_start_amino_acid()
    seed_sequence = list(start_aa)
    generated_peptide = generate_sequence_from_seed(gen_model, seed_sequence, max_length=sampled_length, temperature=0.7, device=device)
    generated_peptides.append(generated_peptide)
    print("Generated AMP sequence:", generated_peptide)

# Save all generated sequences into a text file
with open("generated_peptides.fasta", "w") as f:
    for i, peptide in enumerate(generated_peptides):
        f.write(f">peptide{i}\n")
        f.write(peptide + "\n")

Generated AMP sequence: EWPLADAVLPDYPGGG
Generated AMP sequence: AHGEMCGHIPGKKG
Generated AMP sequence: KAAYYRGDPHGKGYGPHKKG
Generated AMP sequence: IQWVLYAGCHHHPHHKGPGYPGDPPPPGHGPKHGHHPKR
Generated AMP sequence: IAPQKQKKHCGDPHKYGHP
Generated AMP sequence: FADAYRPVRGYK
Generated AMP sequence: EVDADHRTAKHGGHYHKPHYYGHHHG
Generated AMP sequence: VHADWGDKYGPPGPHKKKGP
Generated AMP sequence: DAALPAKGPGPHYG
Generated AMP sequence: WAWTEPRKKGGYYK
Generated AMP sequence: QDPVVRPDGYKCGP
Generated AMP sequence: CAKPPFPGHP
Generated AMP sequence: ASNKAHGGYHYNPKKHGPHG
Generated AMP sequence: DAWYWHAYHYYRHHPHGKYP
Generated AMP sequence: HALYAKVPPPHHPPRYKHPK
Generated AMP sequence: IRVFGALAKKHKFGPHGKPG
Generated AMP sequence: TAAKKNPYKHPHHGGYPPHG
Generated AMP sequence: YAKPPGYPKLHG
Generated AMP sequence: AQAAAATPKGKIGHH
Generated AMP sequence: VLAGMKTPGEPDKYKHKHKY
Generated AMP sequence: HYVAAAADPEHRDPHG
Generated AMP sequence: VLTPRGFPRYHHPHYPHPPG
Generated AMP sequence: QHSFVHAYEHYYKKHHHPPPPGHGK

In [242]:
torch.cuda.is_available() 

False

# TB no transfer

In [ ]:
# Re-import necessary packages after reset
import torch
import torch.nn as nn
import torch.optim as optim
import datetime
import optuna
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, roc_auc_score
from torch.utils.tensorboard import SummaryWriter

from torch.nn.utils.rnn import pad_packed_sequence


class GenerativeLSTM(nn.Module):
    def __init__(self, input_dim=20, hidden_dim=128, num_layers=1, dropout=0.3):
        super(GenerativeLSTM, self).__init__()
        self.lstm = nn.LSTM(
            input_dim, hidden_dim, num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        self.fc = nn.Linear(hidden_dim, input_dim)

    def forward(self, x):
        # Handle packed input
        if isinstance(x, torch.nn.utils.rnn.PackedSequence):
            packed_output, _ = self.lstm(x)
            unpacked_output, _ = pad_packed_sequence(packed_output, batch_first=True)
            return self.fc(unpacked_output)
        else:
            out, _ = self.lstm(x)
            return self.fc(out)

# Criterion

PAD_IDX = -100
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)


def compute_last_token_loss(output, target_seq, criterion):
    """
    Computes cross-entropy loss on the last time step of each sequence.
    
    Args:
        output: Tensor of shape [B, L, vocab_size]
        target_seq: Tensor of shape [B, L] containing target class indices
    
    Returns:
        loss: Scalar loss computed only on the last token of each sequence
    """
    # Get last time step for each sequence
    last_token_logits = output[:, -1, :]        # [B, vocab_size]
    last_token_targets = target_seq[:, -1, :]      # [B]
    last_token_targets = torch.argmax(last_token_targets, dim=-1)  #  now shape is [batch_size, seq_len]

    # print('last_token_logits',last_token_logits.shape)
    # print('last_token_targets',last_token_targets.shape)

    return criterion(last_token_logits, last_token_targets)

# Training function
def train_model_generation(model, train_loader, val_loader, num_epochs=10, lr=1e-3, weight_decay=1e-4,
                           device='cuda' if torch.cuda.is_available() else 'cpu', verbose=False, train=True):
    model.to(device)
    if not train:
        model.eval()

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    best_val_loss = float('inf')
    log_dir = f"runs-lstm-gen-notrans-tb/AMP_LSTM_GEN_{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
    writer = SummaryWriter(log_dir=log_dir)

    for epoch in range(1, num_epochs + 1):
        model.train()
        epoch_loss = 0.0
        for input_seq, target_seq, _ in train_loader:
            input_seq = input_seq.to(device)
            target_seq = target_seq.to(device)

            optimizer.zero_grad()
            output = model(input_seq)
            # output = output.view(-1, output.shape[-1])
            # output = output.reshape(-1, output.shape[-1])      # [B*L, vocab]
            # output = torch.argmax(output, dim=-1)  # now shape is [batch_size, seq_len]

            
            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)
            # # target_seq = target_seq.view(-1)
            # target_seq = torch.argmax(target_seq, dim=-1)  #  now shape is [batch_size, seq_len]

            
            # print('target_shape before reshape',target_seq.shape)
            # target_seq = target_seq.reshape(-1)
            # print(f"Output shape: {output.shape}, Target shape: {target_seq.shape}")

            # loss = criterion(output, target_seq)
            loss = compute_last_token_loss(output, target_seq, criterion)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        avg_train_loss = epoch_loss / len(train_loader)
        val_loss, acc, auc = evaluate_model_generation(model, val_loader, criterion, device, verbose)

        writer.add_scalar('Loss/Train', avg_train_loss, epoch)
        writer.add_scalar('Loss/Validation', val_loss, epoch)
        writer.add_scalar('Accuracy/Validation', acc, epoch)
        writer.add_scalar('AUC/Validation', auc if auc != "undefined" else 0.0, epoch)

        if verbose:
            print(f"Epoch [{epoch}/{num_epochs}] - Train Loss: {avg_train_loss:.4f}, Val Loss: {val_loss:.4f}, Acc: {acc:.4f}, AUC: {auc}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            if train:
                torch.save(model.state_dict(), 'best_model_lstm_generator-notrans-tb.pt')

    writer.close()
    return best_val_loss

# Evaluation function
def evaluate_model_generation(model, data_loader, criterion, device='cuda' if torch.cuda.is_available() else 'cpu', verbose=False):
    model.eval()
    all_labels = []
    all_preds = []
    total_loss = 0.0
    with torch.no_grad():
        for input_seq, target_seq, _ in data_loader:
            input_seq = input_seq.to(device)
            target_seq = target_seq.to(device)

            output = model(input_seq)
            # output = output.view(-1, output.shape[-1])
            # output = output.reshape(-1, output.shape[-1])  # [B*L, vocab]
            # output = torch.argmax(output, dim=-1)  # now shape is [batch_size, seq_len]

            # # target_seq = target_seq.view(-1)
            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)
            # # target_seq = target_seq.view(-1)
            # # target_seq = target_seq.reshape(-1)
            # # target_seq = target_seq.reshape(-1, target_seq.shape[-1])
            # target_seq = torch.argmax(target_seq, dim=-1)  #  now shape is [batch_size, seq_len]

            # assert output.size(0) == target_seq.size(0), f"Mismatch: {output.size(0)} vs {target_seq.size(0)}"

            # loss = criterion(output, target_seq)
            loss = compute_last_token_loss(output, target_seq, criterion)
            # print('loss done')
            total_loss += loss.item()
            
            
            preds = output[:, -1, :]        # shape: [B, vocab_size]
            preds = torch.argmax(preds, dim=1)  # shape: [B]

            targets = target_seq[:, -1, :]      # shape: [B, vocab_size]
            targets = torch.argmax(targets, dim=-1)  # shape: [B]
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    avg_loss = total_loss / len(data_loader)
    acc = accuracy_score(all_labels, all_preds)
    # try:
    #     auc = roc_auc_score(
    #         torch.nn.functional.one_hot(torch.tensor(all_labels), num_classes=20),
    #         torch.nn.functional.one_hot(torch.tensor(all_preds), num_classes=20),
    #         multi_class='ovr', average='macro'
    #     )
    # except:
    auc = "undefined"

    return avg_loss, acc, auc

# Objective for Optuna tuning
def objective_generation(trial):
    hidden_dim = trial.suggest_int("hidden_dim", 64, 256)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    lr = trial.suggest_float("lr", 1e-4, 1e-2)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3)

    model = GenerativeLSTM(hidden_dim=hidden_dim, num_layers=num_layers, dropout=dropout)
    val_loss = train_model_generation(model, train_loader, val_loader, num_epochs=10, lr=lr, weight_decay=weight_decay, verbose=False)
    return val_loss

study = optuna.create_study(direction="minimize")
study.optimize(objective_generation, n_trials=20)

lstm_gen_notrans_tb_best_params = study.best_trial.params
print(lstm_gen_notrans_tb_best_params)

[I 2025-04-23 17:27:16,689] A new study created in memory with name: no-name-fce0a8bc-2cfe-46cc-b056-da4361ab3bf9
[I 2025-04-23 17:27:21,070] Trial 0 finished with value: 2.6234683990478516 and parameters: {'hidden_dim': 227, 'num_layers': 1, 'dropout': 0.46511171984776123, 'lr': 0.007525480141110266, 'weight_decay': 0.0006493437340915416}. Best is trial 0 with value: 2.6234683990478516.
[I 2025-04-23 17:27:26,186] Trial 1 finished with value: 2.7615227699279785 and parameters: {'hidden_dim': 87, 'num_layers': 3, 'dropout': 0.39720828640486194, 'lr': 0.004563633109233991, 'weight_decay': 0.0006372563882503678}. Best is trial 0 with value: 2.6234683990478516.
[I 2025-04-23 17:27:31,797] Trial 2 finished with value: 2.695784568786621 and parameters: {'hidden_dim': 231, 'num_layers': 2, 'dropout': 0.32349721668470843, 'lr': 0.00699099456729401, 'weight_decay': 5.5818143337677604e-05}. Best is trial 0 with value: 2.6234683990478516.
[I 2025-04-23 17:27:36,662] Trial 3 finished with value: 

{'hidden_dim': 125, 'num_layers': 3, 'dropout': 0.1809741409069741, 'lr': 0.009903431049726066, 'weight_decay': 0.00095634187480499}


### test

In [49]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, roc_auc_score
import math

# --- Assumes you already have these from your previous steps ---
# lstm_gen_best_params
# train_loader, val_loader, test_loader
# GenerativeLSTM
# compute_last_token_loss

PAD_IDX = -100
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_final_model(model, train_loader, val_loader, num_epochs=20):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lstm_gen_notrans_tb_best_params["lr"], weight_decay=lstm_gen_notrans_tb_best_params["weight_decay"])
    writer = SummaryWriter(log_dir=f"runs-lstm-gen-notrans-tb/AMPGen_LSTM_final")

    best_val_loss = float('inf')

    for epoch in range(1, num_epochs + 1):
        model.train()
        total_loss = 0.0

        for input_seq, target_seq, _ in train_loader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            optimizer.zero_grad()

            output = model(input_seq)
            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        val_loss, acc, auc, perp = evaluate_final_model(model, test_loader)

        writer.add_scalar('Loss/Train', avg_train_loss, epoch)
        writer.add_scalar('Loss/Validation', val_loss, epoch)
        writer.add_scalar('Accuracy/Validation', acc, epoch)
        writer.add_scalar('AUC/Validation', auc if auc != "undefined" else 0.0, epoch)

        print(f"Epoch {epoch}: Train Loss = {avg_train_loss:.4f} | Val Loss = {val_loss:.4f} | Acc = {acc:.4f} | AUC = {auc} | Perplexity = {perp:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            # torch.save(model.state_dict(), "best_model_lstm_generator.pt")

    writer.close()
    return model

def evaluate_final_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    total_loss = 0.0

    with torch.no_grad():
        for input_seq, target_seq, _ in data_loader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            output = model(input_seq)

            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            total_loss += loss.item()

            preds = output[:, -1, :]  # [B, vocab]
            preds = torch.argmax(preds, dim=1)
            targets = target_seq[:, -1, :]
            targets = torch.argmax(targets, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    # try:
    #     auc = roc_auc_score(
    #         torch.nn.functional.one_hot(torch.tensor(all_labels), num_classes=20),
    #         torch.nn.functional.one_hot(torch.tensor(all_preds), num_classes=20),
    #         multi_class='ovr', average='macro'
    #     )
    # except Exception:
    auc = "undefined"

    avg_loss = total_loss / len(data_loader)
    perplexity = math.exp(avg_loss)
    return avg_loss, acc, auc, perplexity

# --- Build and train final model using best parameters ---

final_model = GenerativeLSTM(
    input_dim=20,
    hidden_dim=lstm_gen_notrans_tb_best_params["hidden_dim"],
    num_layers=lstm_gen_notrans_tb_best_params["num_layers"],
    dropout=lstm_gen_notrans_tb_best_params["dropout"]
)

trained_model = train_final_model(final_model, train_loader, val_loader, num_epochs=60)

# --- Evaluate on test set ---
test_loss, test_acc, test_auc, perp = evaluate_final_model(trained_model, test_loader)
print(f"\n✅ Final Test Metrics:\nLoss = {test_loss:.4f}, Accuracy = {test_acc:.4f}, AUC = {test_auc}, Perplexity = {perp:.4f}")


Epoch 1: Train Loss = 2.9141 | Val Loss = 2.8729 | Acc = 0.9730 | AUC = undefined | Perplexity = 17.6879
Epoch 2: Train Loss = 2.8598 | Val Loss = 2.8129 | Acc = 0.9730 | AUC = undefined | Perplexity = 16.6579
Epoch 3: Train Loss = 2.7946 | Val Loss = 2.7297 | Acc = 1.0000 | AUC = undefined | Perplexity = 15.3288
Epoch 4: Train Loss = 2.7253 | Val Loss = 2.6674 | Acc = 0.9730 | AUC = undefined | Perplexity = 14.4023
Epoch 5: Train Loss = 2.6725 | Val Loss = 2.6059 | Acc = 1.0000 | AUC = undefined | Perplexity = 13.5428
Epoch 6: Train Loss = 2.6100 | Val Loss = 2.5701 | Acc = 0.9730 | AUC = undefined | Perplexity = 13.0667
Epoch 7: Train Loss = 2.5629 | Val Loss = 2.5236 | Acc = 0.9730 | AUC = undefined | Perplexity = 12.4729
Epoch 8: Train Loss = 2.5047 | Val Loss = 2.4722 | Acc = 0.9730 | AUC = undefined | Perplexity = 11.8488
Epoch 9: Train Loss = 2.4530 | Val Loss = 2.4157 | Acc = 0.9730 | AUC = undefined | Perplexity = 11.1979
Epoch 10: Train Loss = 2.4267 | Val Loss = 2.3613 | Acc

### batch gen

In [50]:
import random
import numpy as np

# Set global seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Define the amino acid vocabulary
amino_acids = "ACDEFGHIKLMNPQRSTVWY"

def sample_start_amino_acid():
    return random.choice(amino_acids)

# Reload the trained model
gen_model = trained_model
gen_model.to(device)

generated_peptides = []
# (Re-run the generation loop to collect sequences)
for x in range(100):
    sampled_length = length_sampler.sample()[0]
    # sampled_length = 20
    start_aa = sample_start_amino_acid()
    seed_sequence = list(start_aa)
    generated_peptide = generate_sequence_from_seed(gen_model, seed_sequence, max_length=sampled_length, temperature=0.7, device=device)
    generated_peptides.append(generated_peptide)
    print("Generated AMP sequence:", generated_peptide)

# Save all generated sequences into a text file
with open("generated_peptides-notrans.fasta", "w") as f:
    for i, peptide in enumerate(generated_peptides):
        f.write(f">peptide{i}\n")
        f.write(peptide + "\n")

Generated AMP sequence: EVKKDGEPKKDYKHGG
Generated AMP sequence: AAADKGHKKKGKKG
Generated AMP sequence: KAAYYPHGKKGKGPHKHKKG
Generated AMP sequence: IHWTKYGGGHKHKKHKGKGYKGDKKKKGKGPKKGKKKKP
Generated AMP sequence: IAKPKPKKKEGGKHKPHKK
Generated AMP sequence: FAAAYPKPMGPK
Generated AMP sequence: EQAAGKPPAKKHGKYKKKKYYHKHHG
Generated AMP sequence: VAADYGGKYHKKHKKKKKGK
Generated AMP sequence: DAAKKAKGKGKKYG
Generated AMP sequence: WAVSGKPKKHGPPK
Generated AMP sequence: QAKVVPKGGPKDGK
Generated AMP sequence: CAGLKGKHHK
Generated AMP sequence: AMIKAHHHYKPKKKKKGKHG
Generated AMP sequence: DAWYYHAYKYYPHKPKGKYK
Generated AMP sequence: HAHYDKRKKKKHKKPPKKKK
Generated AMP sequence: IKSFHEKDKKHKGGKKGKKH
Generated AMP sequence: TAAIKKKYKHKHKGGYKKKG
Generated AMP sequence: YAGMKHYKKKKH
Generated AMP sequence: AGAAAAPKKGKKGKK
Generated AMP sequence: VAAGKKPKGGKGKYKKKKKY
Generated AMP sequence: HVTAAFAGKGHPGKHG
Generated AMP sequence: VARPPHGKPYKKKKYPKKKG
Generated AMP sequence: QAPEVHATGHYYKKKHKKKKKGKGK

# production

In [41]:
from torch.nn.utils.rnn import pad_packed_sequence


class GenerativeLSTM(nn.Module):
    def __init__(self, input_dim=20, hidden_dim=128, num_layers=1, dropout=0.3):
        super(GenerativeLSTM, self).__init__()
        self.lstm = nn.LSTM(
            input_dim, hidden_dim, num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        self.fc = nn.Linear(hidden_dim, input_dim)

    def forward(self, x):
        # Handle packed input
        if isinstance(x, torch.nn.utils.rnn.PackedSequence):
            packed_output, _ = self.lstm(x)
            unpacked_output, _ = pad_packed_sequence(packed_output, batch_first=True)
            return self.fc(unpacked_output)
        else:
            out, _ = self.lstm(x)
            return self.fc(out)


### no trans - test

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, roc_auc_score
import math

# --- Assumes you already have these from your previous steps ---
# lstm_gen_best_params
# train_loader, val_loader, test_loader
# GenerativeLSTM
# compute_last_token_loss

PAD_IDX = -100
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def compute_last_token_loss(output, target_seq, criterion):
    """
    Computes cross-entropy loss on the last time step of each sequence.
    
    Args:
        output: Tensor of shape [B, L, vocab_size]
        target_seq: Tensor of shape [B, L] containing target class indices
    
    Returns:
        loss: Scalar loss computed only on the last token of each sequence
    """
    # Get last time step for each sequence
    last_token_logits = output[:, -1, :]        # [B, vocab_size]
    last_token_targets = target_seq[:, -1, :]      # [B]
    last_token_targets = torch.argmax(last_token_targets, dim=-1)  #  now shape is [batch_size, seq_len]

    # print('last_token_logits',last_token_logits.shape)
    # print('last_token_targets',last_token_targets.shape)

    return criterion(last_token_logits, last_token_targets)


def train_final_model(model, train_loader, val_loader, num_epochs=20):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lstm_gen_notrans_tb_best_params["lr"], weight_decay=lstm_gen_notrans_tb_best_params["weight_decay"])
    writer = SummaryWriter(log_dir=f"runs-lstm-gen-notrans-tb/AMPGen_LSTM_final")

    best_val_loss = float('inf')

    for epoch in range(1, num_epochs + 1):
        model.train()
        total_loss = 0.0

        for input_seq, target_seq, _ in train_loader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            optimizer.zero_grad()

            output = model(input_seq)
            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        val_loss, acc, auc, perp = evaluate_final_model(model, test_loader)

        writer.add_scalar('Loss/Train', avg_train_loss, epoch)
        writer.add_scalar('Loss/Validation', val_loss, epoch)
        writer.add_scalar('Accuracy/Validation', acc, epoch)
        writer.add_scalar('AUC/Validation', auc if auc != "undefined" else 0.0, epoch)

        print(f"Epoch {epoch}: Train Loss = {avg_train_loss:.4f} | Val Loss = {val_loss:.4f} | Acc = {acc:.4f} | AUC = {auc} | Perplexity = {perp:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            # torch.save(model.state_dict(), "best_model_lstm_generator.pt")

    writer.close()
    return model

def evaluate_final_model(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    total_loss = 0.0

    with torch.no_grad():
        for input_seq, target_seq, _ in data_loader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            output = model(input_seq)

            if isinstance(target_seq, torch.nn.utils.rnn.PackedSequence):
                target_seq, _ = pad_packed_sequence(target_seq, batch_first=True)

            loss = compute_last_token_loss(output, target_seq, criterion)
            total_loss += loss.item()

            preds = output[:, -1, :]  # [B, vocab]
            preds = torch.argmax(preds, dim=1)
            targets = target_seq[:, -1, :]
            targets = torch.argmax(targets, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    # try:
    #     auc = roc_auc_score(
    #         torch.nn.functional.one_hot(torch.tensor(all_labels), num_classes=20),
    #         torch.nn.functional.one_hot(torch.tensor(all_preds), num_classes=20),
    #         multi_class='ovr', average='macro'
    #     )
    # except Exception:
    auc = "undefined"

    avg_loss = total_loss / len(data_loader)
    perplexity = math.exp(avg_loss)
    return avg_loss, acc, auc, perplexity

# --- Build and train final model using best parameters ---
lstm_gen_notrans_tb_best_params = {'hidden_dim': 125, 'num_layers': 3, 'dropout': 0.1809741409069741, 'lr': 0.009903431049726066, 'weight_decay': 0.00095634187480499}

final_model = GenerativeLSTM(
    input_dim=20,
    hidden_dim=lstm_gen_notrans_tb_best_params["hidden_dim"],
    num_layers=lstm_gen_notrans_tb_best_params["num_layers"],
    dropout=lstm_gen_notrans_tb_best_params["dropout"]
)

trained_model = train_final_model(final_model, train_loader, val_loader, num_epochs=60)

# --- Evaluate on test set ---
test_loss, test_acc, test_auc, perp = evaluate_final_model(trained_model, test_loader)
print(f"\n✅ Final Test Metrics:\nLoss = {test_loss:.4f}, Accuracy = {test_acc:.4f}, AUC = {test_auc}, Perplexity = {perp:.4f}")
# torch.save(trained_model.state_dict(), "final_amp_notrans_generator_lstm.pt")


Epoch 1: Train Loss = 2.9193 | Val Loss = 2.8783 | Acc = 0.9730 | AUC = undefined | Perplexity = 17.7838
Epoch 2: Train Loss = 2.8637 | Val Loss = 2.8221 | Acc = 0.9730 | AUC = undefined | Perplexity = 16.8114
Epoch 3: Train Loss = 2.8002 | Val Loss = 2.7504 | Acc = 0.9730 | AUC = undefined | Perplexity = 15.6490
Epoch 4: Train Loss = 2.7289 | Val Loss = 2.6782 | Acc = 0.9730 | AUC = undefined | Perplexity = 14.5591
Epoch 5: Train Loss = 2.6731 | Val Loss = 2.6309 | Acc = 0.9730 | AUC = undefined | Perplexity = 13.8861
Epoch 6: Train Loss = 2.6404 | Val Loss = 2.5762 | Acc = 0.9730 | AUC = undefined | Perplexity = 13.1466
Epoch 7: Train Loss = 2.5704 | Val Loss = 2.5244 | Acc = 0.9730 | AUC = undefined | Perplexity = 12.4831
Epoch 8: Train Loss = 2.5093 | Val Loss = 2.4685 | Acc = 0.9730 | AUC = undefined | Perplexity = 11.8052
Epoch 9: Train Loss = 2.5077 | Val Loss = 2.4155 | Acc = 0.9730 | AUC = undefined | Perplexity = 11.1951
Epoch 10: Train Loss = 2.4048 | Val Loss = 2.3679 | Acc

#### batch gen

In [ ]:
import random
import numpy as np
from collections import Counter
import torch
import torch.nn.functional as F
import numpy as np

# Set global seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# final_model = GenerativeLSTM(
#     input_dim=20,
#     hidden_dim=lstm_gen_notrans_tb_best_params["hidden_dim"],
#     num_layers=lstm_gen_notrans_tb_best_params["num_layers"],
#     dropout=lstm_gen_notrans_tb_best_params["dropout"]
# )
# trained_model = GenerativeLSTM(
#     input_dim=20,
#     hidden_dim=176,
#     num_layers=lstm_gen_notrans_tb_best_params["num_layers"],
#     dropout=lstm_gen_notrans_tb_best_params["dropout"]
# )


# final_model.load_state_dict(torch.load("best_model_lstm_generator-notrans-tb.pt"))

# Define the amino acid vocabulary
amino_acids = "ACDEFGHIKLMNPQRSTVWY"

def sample_start_amino_acid():
    return random.choice(amino_acids)

class LengthSampler:
    def __init__(self, sequence_lengths):
        """
        Initialize sampler from observed sequence lengths.
        
        Args:
            sequence_lengths (list[int]): List of sequence lengths (e.g., [20, 21, 20, 23, ...])
        """
        self.length_counts = Counter(sequence_lengths)
        self.lengths = np.array(sorted(self.length_counts.keys()))
        counts = np.array([self.length_counts[l] for l in self.lengths])
        self.probs = counts / counts.sum()  # Empirical probabilities

    def sample(self, n=1):
        """
        Sample one or more lengths based on the learned distribution.

        Returns:
            np.ndarray of sampled lengths
        """
        return np.random.choice(self.lengths, size=n, p=self.probs)
length_sampler = LengthSampler([len(seq) for seq in df.loc[df['AMP'] == 1, :]['Sequences']])


# Recreate your amino acid vocab
aa_vocab = 'ACDEFGHIKLMNPQRSTVWY'
aa_to_idx = {aa: i for i, aa in enumerate(aa_vocab)}
idx_to_aa = {i: aa for aa, i in aa_to_idx.items()}

def one_hot_encode_amino_acid(aa, vocab=aa_vocab):
    vec = torch.zeros(len(vocab))
    vec[aa_to_idx[aa]] = 1.0
    return vec

def generate_sequence_from_seed(model, seed, max_length=30, temperature=1.0, device='cpu'):
    model.eval()
    input_seq = [one_hot_encode_amino_acid(aa).to(device) for aa in seed]
    input_tensor = torch.stack(input_seq).unsqueeze(0)  # [1, L, 20]

    generated = seed.copy()

    with torch.no_grad():
        for _ in range(max_length - len(seed)):
            output = model(input_tensor)  # [1, L, vocab]
            logits = output[0, -1, :]  # Last time step → [vocab]

            # Apply temperature and sample
            probs = F.softmax(logits / temperature, dim=-1).cpu().numpy()
            next_idx = np.random.choice(len(aa_vocab), p=probs)
            next_aa = idx_to_aa[next_idx]

            # Update sequence
            next_aa_vec = one_hot_encode_amino_acid(next_aa).to(device).unsqueeze(0).unsqueeze(0)  # [1, 1, 20]
            input_tensor = torch.cat([input_tensor, next_aa_vec], dim=1)
            generated.append(next_aa)

    return ''.join(generated)

# Reload the trained model
gen_model = trained_model
gen_model.to(device)


generated_peptides = []
# (Re-run the generation loop to collect sequences)
for x in range(100):
    sampled_length = length_sampler.sample()[0]
    # sampled_length = 20
    start_aa = sample_start_amino_acid()
    seed_sequence = list(start_aa)
    generated_peptide = generate_sequence_from_seed(gen_model, seed_sequence, max_length=sampled_length, temperature=1, device=device)
    generated_peptides.append(generated_peptide)
    print("Generated AMP sequence:", generated_peptide)
model
# Save all generated sequences into a text file
with open("generated_peptides-notrans.fasta", "w") as f:
    for i, peptide in enumerate(generated_peptides):
        f.write(f">peptide{i}\n")
        f.write(peptide + "\n")

## No trans

In [42]:
import random
import numpy as np

# Set global seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Define the amino acid vocabulary
amino_acids = "ACDEFGHIKLMNPQRSTVWY"

def sample_start_amino_acid():
    return random.choice(amino_acids)

lstm_gen_best_params = {'hidden_dim': 125, 'num_layers': 3, 'dropout': 0.1809741409069741, 'lr': 0.009903431049726066, 'weight_decay': 0.00095634187480499}

# lstm_gen_best_params = {'dropout': 0.13882938162931305, 'lr': 0.001464264209101335, 'weight_decay': 0.0002912839503135125}

# Reload the trained model
gen_model = GenerativeLSTM(
    input_dim=20,
    hidden_dim=lstm_gen_best_params["hidden_dim"],
    num_layers=lstm_gen_best_params["num_layers"],
    dropout=lstm_gen_best_params["dropout"]
)
gen_model.load_state_dict(torch.load("final_amp_notrans_generator_lstm.pt"))
gen_model.to(device)

generated_peptides = []
# (Re-run the generation loop to collect sequences)
for x in range(500):
    sampled_length = length_sampler.sample()[0]
    # sampled_length = 20
    start_aa = sample_start_amino_acid()
    seed_sequence = list(start_aa)
    generated_peptide = generate_sequence_from_seed(gen_model, seed_sequence, max_length=sampled_length, temperature=1, device=device)
    generated_peptides.append(generated_peptide)
    print("Generated AMP sequence:", generated_peptide)

# Save all generated sequences into a text file
with open("generated_peptides-notrans1-500.fasta", "w") as f:
    for i, peptide in enumerate(generated_peptides):
        f.write(f">peptide{i}\n")
        f.write(peptide + "\n")

Generated AMP sequence: EWPKCECWKPCYSGGG
Generated AMP sequence: AEFDLDGHKQGKKD
Generated AMP sequence: KAAYYSHENKGKDYGKHKKG
Generated AMP sequence: IPWVKYDGDHHHSHHKGRGYPGASNPPGHGYKHEHHPKY
Generated AMP sequence: IANQKQKKKCGDKHKYGHP
Generated AMP sequence: FAAAYSLWRGYK
Generated AMP sequence: ETAAEHSVAKIGGHYHKMHYYGKHHD
Generated AMP sequence: VDADWGGKYGKPGPHKKKGS
Generated AMP sequence: DAAKPAKGKGMHYG
Generated AMP sequence: WAWTGNSKKGGYYK
Generated AMP sequence: QANVVRLDGYKAGK
Generated AMP sequence: CAHNMGPGHP
Generated AMP sequence: ARLKAHGGYHYKQKKKGPHD
Generated AMP sequence: DAWYYHAYKYYVHHTHGKYL
Generated AMP sequence: HAKYAKWPPPHHRRYYKKQK
Generated AMP sequence: IPTFHCKAKKHKDDSHGKPG
Generated AMP sequence: TAAKKLPYKHQHKGDYSLHG
Generated AMP sequence: YAHPNGYPKKIG
Generated AMP sequence: ANAAAATPKGKKGKH
Generated AMP sequence: VIAGLKTLGDKDKYKHKKKY
Generated AMP sequence: HWVAACADNDHSDSHG
Generated AMP sequence: VHTPRGGPSYHHPHYWKPPG
Generated AMP sequence: QDRFWHAYFHYYKKKHHKPPPGKDK

## Frozen

In [15]:
import random
import numpy as np

# Set global seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Define the amino acid vocabulary
amino_acids = "ACDEFGHIKLMNPQRSTVWY"

def sample_start_amino_acid():
    return random.choice(amino_acids)

lstm_gen_best_params = {'hidden_dim': 247, 'num_layers': 3, 'dropout': 0.12915638774083413, 'lr': 0.009983517577195865, 'weight_decay': 0.0007901836295730717}

# lstm_gen_best_params = {'dropout': 0.13882938162931305, 'lr': 0.001464264209101335, 'weight_decay': 0.0002912839503135125}

# Reload the trained model
gen_model = GenerativeLSTM(
    input_dim=20,
    hidden_dim=lstm_gen_best_params["hidden_dim"],
    num_layers=lstm_gen_best_params["num_layers"],
    dropout=lstm_gen_best_params["dropout"]
)
gen_model.load_state_dict(torch.load("final_amp_frozen_generator_lstm.pt"))
gen_model.to(device)

generated_peptides = []
# (Re-run the generation loop to collect sequences)
for x in range(100):
    sampled_length = length_sampler.sample()[0]
    # sampled_length = 20
    start_aa = sample_start_amino_acid()
    seed_sequence = list(start_aa)
    generated_peptide = generate_sequence_from_seed(gen_model, seed_sequence, max_length=sampled_length, temperature=1, device=device)
    generated_peptides.append(generated_peptide)
    print("Generated AMP sequence:", generated_peptide)

# Save all generated sequences into a text file
with open("generated_peptides-.fasta", "w") as f:
    for i, peptide in enumerate(generated_peptides):
        f.write(f">peptide{i}\n")
        f.write(peptide + "\n")

Generated AMP sequence: EWPLAAATLPAYSDDD
Generated AMP sequence: AIHFMAGGHRDKLA
Generated AMP sequence: KAAYYSGAPHAIAWFNGKKD
Generated AMP sequence: IQWVLWADAGHFSGFKASAYRDASPPRAGATMGAGGPNV
Generated AMP sequence: IAPQKRIKHAAANGKWEHQ
Generated AMP sequence: FAEAWSNTSDVK
Generated AMP sequence: EVEADHSTAKHDAGYGKPGYYEIGFA
Generated AMP sequence: VHAEWEAIYEPQEPGMNKAS
Generated AMP sequence: DAALPAKDNCPHYA
Generated AMP sequence: WAWVENSKKEAVVN
Generated AMP sequence: QEPVVRNACVMAAP
Generated AMP sequence: CAKPNDPEGQ
Generated AMP sequence: ASNKAGFEYHVMRKLIDPFA
Generated AMP sequence: DAWYWGAWHYYTGGTGCKYP
Generated AMP sequence: HAMYAKVQPPGGSSTWKKRN
Generated AMP sequence: IRVGGALAHKGLAASGAKRD
Generated AMP sequence: TAAKKNPYKGRFHAAYSPHC
Generated AMP sequence: YAKPNFYQKMHE
Generated AMP sequence: AQAAAATPIAIICHH
Generated AMP sequence: VMAGMKTNCANALYLHNHKY
Generated AMP sequence: HYVCAAAAPAGTASFA
Generated AMP sequence: VMTQSFCQSYHGRGWTHQQA
Generated AMP sequence: QHSFVGAWAGYYLMHGGPQRRAIAK

## Full backprop

In [16]:
import random
import numpy as np

# Set global seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Define the amino acid vocabulary
amino_acids = "ACDEFGHIKLMNPQRSTVWY"

def sample_start_amino_acid():
    return random.choice(amino_acids)

lstm_gen_best_params = {'hidden_dim': 247, 'num_layers': 3, 'dropout': 0.12915638774083413, 'lr': 0.009983517577195865, 'weight_decay': 0.0007901836295730717}

# lstm_gen_best_params = {'dropout': 0.13882938162931305, 'lr': 0.001464264209101335, 'weight_decay': 0.0002912839503135125}

# Reload the trained model
gen_model = GenerativeLSTM(
    input_dim=20,
    hidden_dim=lstm_gen_best_params["hidden_dim"],
    num_layers=lstm_gen_best_params["num_layers"],
    dropout=lstm_gen_best_params["dropout"]
)
gen_model.load_state_dict(torch.load("final_amp_generator_lstm.pt"))
gen_model.to(device)

generated_peptides = []
# (Re-run the generation loop to collect sequences)
for x in range(100):
    sampled_length = length_sampler.sample()[0]
    # sampled_length = 20
    start_aa = sample_start_amino_acid()
    seed_sequence = list(start_aa)
    generated_peptide = generate_sequence_from_seed(gen_model, seed_sequence, max_length=sampled_length, temperature=1, device=device)
    generated_peptides.append(generated_peptide)
    print("Generated AMP sequence:", generated_peptide)

# Save all generated sequences into a text file
with open("generated_peptides-fullback1.fasta", "w") as f:
    for i, peptide in enumerate(generated_peptides):
        f.write(f">peptide{i}\n")
        f.write(peptide + "\n")

Generated AMP sequence: EWQMDDAVNPAYRGGG
Generated AMP sequence: AKHFNDGHKRGKKD
Generated AMP sequence: KAAYYSGDPIGKDYHPHKKG
Generated AMP sequence: IRWVMYCGAHHHRHHKGPGYPGDPPPPGHGRKHGHHPLY
Generated AMP sequence: IAQRLRKKIAGDPHKYGHP
Generated AMP sequence: FAFCYSPVRGYK
Generated AMP sequence: EVFAEISTAKHGGHYHKPHYYGKHHG
Generated AMP sequence: VKAFWGDKYGPPGPHPPKGP
Generated AMP sequence: DAAMPAKGPGPHYG
Generated AMP sequence: WAWVFPSLKGFYYP
Generated AMP sequence: QFQVVRPDGYNAGP
Generated AMP sequence: CALPPFPGHP
Generated AMP sequence: ATPLAHGGYHYPPKKKGPHD
Generated AMP sequence: DAWYWHAYIYYSHHSHGKYP
Generated AMP sequence: HANYCKVQPPHHQQWYKKPP
Generated AMP sequence: ISVGHAMAKKHLDDQHGKPG
Generated AMP sequence: TAAKLPQYKHRHHGDYQPHG
Generated AMP sequence: YDLQPGYQKPHG
Generated AMP sequence: ARAAAATPKEKKGHH
Generated AMP sequence: VNAHNKTPGDPDKYKHPHKY
Generated AMP sequence: HYVDAAADPDHSDQHG
Generated AMP sequence: VNVQSGEQRYHHPHYSHPPG
Generated AMP sequence: QKSGVHAYDHYYKPHHHPPPPGKGK